In [1]:
# config
route = '87'
timed_course_start = '21062' # 87 palisade and south
timed_course_end ='20505' # 87 paterson plank and harrison
period = 'today'

In [2]:
# import libraries
import datetime, time, sys
from operator import itemgetter
import pandas as pd

# import app libraries
import reportcard.lib.StopsDB as StopsDB
import reportcard.lib.BusAPI as BusAPI

In [3]:
# database initialization
db = StopsDB.MySQL('buses', 'buswatcher', 'njtransit', '127.0.0.1', route)
conn = db.conn
table_name = 'stop_approaches_log_' + route

In [4]:
# common functions
def timestamp_fix(data): # trim the microseconds off the timestamp and convert it to datetime format
    data['timestamp'] = data['timestamp'].str.split('.').str.get(0)
    data['timestamp'] = pd.to_datetime(data['timestamp'],errors='coerce')
    data = data.set_index(pd.DatetimeIndex(data['timestamp']))
    # data = data.set_index(pd.DatetimeIndex(data['timestamp'], drop=False)
    return data

In [5]:
# get data and basic cleanup
def get_arrivals(route,stop,final_approach_query,conn):
    df_temp = pd.read_sql_query(final_approach_query, conn) # arrivals table and deltas are all re-generated on the fly for every view now -- easier, but might lead to inconsistent/innaccurate results over time?
    df_temp = df_temp.drop(columns=['cars', 'consist', 'fd', 'm', 'name', 'rn', 'scheduled'])
    df_temp = timestamp_fix(df_temp)

    # split final approach history (sorted by timestamp) at each change in vehicle_id outputs a list of dfs -- per https://stackoverflow.com/questions/41144231/python-how-to-split-pandas-dataframe-into-subsets-based-on-the-value-in-the-fir
    final_approach_dfs = [g for i, g in df_temp.groupby(df_temp['v'].ne(df_temp['v'].shift()).cumsum())]

    try:
        # take the last V(ehicle) approach in each df and add it to final list of arrivals
        arrivals_list_final_df = pd.DataFrame()
        for final_approach in final_approach_dfs:  # iterate over every final approach
            arrival_insert_df = final_approach.tail(1)  # take the last observation
            arrivals_list_final_df = arrivals_list_final_df.append(arrival_insert_df)  # insert into df

        # calc interval between last bus for each row, fill NaNs
        arrivals_list_final_df['delta']=(arrivals_list_final_df['timestamp'] - arrivals_list_final_df['timestamp'].shift(-1)).fillna(0)
        # housekeeping ---------------------------------------------------

        # set stop_name
        stop_name = arrivals_list_final_df['stop_name'].iloc[0]
        return arrivals_list_final_df, stop_name

    except:
        arrivals_list_final_df=\
            pd.DataFrame(\
                columns=['pkey','pt','rd','stop_id','stop_name','v','timestamp','delta'],\
                data=[['0000000', '3', route, stop,'N/A', 'N/A', datetime.time(0,1), datetime.timedelta(seconds=0)]])
        stop_name = 'N/A'
        arrivals_table_time_created = datetime.datetime.now()

In [6]:
# select query based on period

arrivals = []

for stop in [timed_course_start,timed_course_end]:
    if period == "daily":
            final_approach_query = ('SELECT * FROM %s WHERE (stop_id= %s AND DATE(`timestamp`)=CURDATE() ) ORDER BY timestamp DESC;' % (table_name, stop))
    elif period == "yesterday":
        final_approach_query = ('SELECT * FROM %s WHERE (stop_id= %s AND (timestamp >= CURDATE() - INTERVAL 1 DAY AND timestamp < CURDATE())) ORDER BY timestamp DESC;' % (table_name, stop))
    elif period=="weekly":
        final_approach_query = ('SELECT * FROM %s WHERE (stop_id= %s AND (YEARWEEK(`timestamp`, 1) = YEARWEEK(CURDATE(), 1))) ORDER BY timestamp DESC;' % (table_name, stop))
    elif period=="history":
        final_approach_query = ('SELECT * FROM %s WHERE stop_id= %s ORDER BY timestamp DESC;' % (table_name,stop))
    else:
        raise RuntimeError('Bad request sucker!')
    
    arrivals.append(get_arrivals(route,stop,final_approach_query,conn))


RuntimeError: Bad request sucker!

## below will work better if i group the data by day first

In [ ]:
# match up trips ---- to create journeys

hill_runs=[]
    
# populate list of runs with buses from top of hill (bus#,starttime)
for index, row in arrivals[0][0].iterrows():
    run_entry = []
    run_entry.append(row['v'])
    run_entry.append(row['timestamp'])
    hill_runs.append(run_entry)

hill_runs

In [ ]:
for index, row in arrivals[1][0].iterrows():
    for bus in hill_runs:
        if (bus[0] == row['v']) and (row['delta'].seconds < 3600): # Error: string indices must be integers
            bus.append(row['timestamp']) # (bus#,starttime,endtime)

hill_runs

In [ ]:
for row in hill_runs:
    duration = row[3] - row[2]
    row.append(duration) # (bus#,starttime,endtime,duration)

hill_runs

In [ ]:
# turn into a data frame then
# resample by hour, mean(travel_time)